# S3準拠オブジェクトストレージを大容量データの保管場所として使う

このNotebookは[AWS S3](https://aws.amazon.com/jp/s3/)に準拠したオブジェクトストレージを大容量データの保管場所として利用する際のサポートを提供します。

## データガバナンス機能における大容量データの管理について

データガバナンス機能では、大容量データの管理にあたって、データファイルのメタデータ（ファイル名、作成日時、実データの所在等の情報）と実データとを分けて管理することができる技術「git-annex」を採用しています。これにより、目下必要となる実データのみをこの実験環境にダウンロードできるため、データ利用の利便性が向上するメリットがあります。このNotebookでは、この管理手法を用いた実データの操作をサポートします。

## このNotebookでできること（ユースケース例を基に）

### 例①　データをgit-annex管理下へ移管する

![usecase_annex_s3_1](https://raw.githubusercontent.com/NII-DG/workflow-template/main/PACKAGE/base/EX-WORKFLOW/images/usecase_annex_s3_1.png)

### 例②　git-annex管理下のデータを利用する

![usecase_annex_s3_2](https://raw.githubusercontent.com/NII-DG/workflow-template/main/PACKAGE/base/EX-WORKFLOW/images/usecase_annex_s3_2.png)

## ０．オブジェクトストレージとの接続設定をする

データ保管の場所として、オブジェクトストレージをシステムが認識するための設定を行います。

※ここでは、適切なキーとシークレットアクセスキーとのIDが必要です。詳しくは、各オブジェクトストレージのドキュメントをご覧いただくか、各管理者にお問い合わせください。

In [ ]:
import os
from IPython.display import clear_output

# 将来的にはGRDM等を利用したキー情報管理を検討する
os.environ['AWS_ACCESS_KEY_ID'] = input('アクセスキーID: ')
os.environ['AWS_SECRET_ACCESS_KEY'] = input('シークレットアクセスキー: ')

clear_output()

In [ ]:
import ipywidgets as widgets
from IPython.display import display

def on_click_callback(clicked_button: widgets.Button) -> str:
    with output:
        output.clear_output()
        # セレクタで選択されているアイテムを使う
        print(f'{select.value}が選択されました。')
        print('次のセルでは、このオブジェクトストレージへの接続処理を進めます。')
        print('選択したストレージでよろしければ、次のセルの実行へお進みください。')
        print('もし選び直す場合は、もう一度上のボックスから選択した後、「決定」ボタンを押してください。')
        
print('どのオブジェクトストレージ(S3)を使いますか？')
print('\t※AWS: Amazon Web Service上で稼働するストレージ\n\t  mdx: mdx環境上で稼働するストレージ')

list_s3 = ['AWS', 'mdx']

# select.valueの値を次のセルで処理の場合分けに利用する
select = widgets.Select(options=list_s3)
button = widgets.Button(description='決　定')
button.on_click(on_click_callback)

output = widgets.Output(layour={'border': '1px solid black'})

display(select, button, output)

In [ ]:
print('データをアップロード／ダウンロードしたいオブジェクトストレージのバケット名を入力してください。')
print('入力された名前のバケットがなければ作成され、あればその既存のバケットが利用されます。')
bucket_name = input('バケット名：')

In [ ]:
# Note: 前までのセルから、select.valueとbucket_nameとを利用している

import time

# 次のセルでも利用する変数のため、関数の外で宣言している
special_remote_name = 'remote_' + select.value + '_' + bucket_name

def connect_s3_storage():
    print('接続設定中です...')
    try:
        msg = ''
        if select.value == 'AWS':
            # for Amazon Web Service(AWS) S3
            msg = !git-annex initremote $special_remote_name type=S3 bucket=$bucket_name encryption=none autoenable=true
        elif select.value == 'mdx':
            # for mdx(s3ds.mdx.jp) S3
            msg = !git-annex initremote $special_remote_name type=S3 port=443 host=s3ds.mdx.jp bucket=$bucket_name encryption=none signature=v2 requeststyle=path autoenable=true
        else:
            raise ValueError('select.value is wrong: ' + select.value) 
            
        if msg[0] in 'There is already a special remote':
            !git-annex enableremote $special_remote_name
        print(select.value + 'への接続設定が完了しました。')
    except:
        print('処理を完了できませんでした。\n利用するオブジェクトストレージが選択されていない可能性があります。')
        print('前のセルを実行いただき、利用したいオブジェクトストレージを選んでください。')
        print('もし選択してもこのメッセージが表示されている場合、恐れ入りますがデータガバナンス機能ヘルプセンター（仮）へお問い合わせください。')
        return

if __name__ == '__main__':
    connect_s3_storage()

## 1-A. 実データをオブジェクトストレージへ保存する

この手順では、任意のデータを「git-annex」による管理下に置き、「０．オブジェクトストレージとの接続設定をする」で設定したオブジェクトストレージへ、実データをアップロードする処理のサポートを提供します。

以下のセルを順番に実行し、表示にしたがって操作することでデータをアップロードできます。

In [ ]:
# データフォルダ内のフォルダ構造を取得し、どれをannex管理するか決める
import os
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display

from scripts import utils

####################################################################################################
########################################## callback functions ######################################
####################################################################################################
def on_click_callback(clicked_button: widgets.Button) -> None:
    with output:
        output.clear_output()
        # セレクタで選択されているアイテムを使う
        print(f'選択されたファイル：{upload_selects.value}')
        print('これらのファイルをオブジェクトストレージへ保存します。問題ない場合は、次のセルへ進んでください。')
        print('選び直す場合は、再度ボックスから選択し、「決定」ボタンを押してください。')
        
def on_click_callback_ALL(clicked_button: widgets.Button):
    with output:
        print('上のボックスに表示されている、全てのファイルをオブジェクトストレージへ保存します。\n問題ない場合は、次のセルへ進んでください。')
        print('選び直す場合は、再度ボックスから選択し、「決定」ボタンを押してください。')
        upload_selects.value = data_files

####################################################################################################
####################################################################################################
####################################################################################################

print('どのデータをアップロードしますか？')
print('　◆◆◆ctrlキー(MAC:commandキー）を押しながらクリックすることで、複数データを選択できます◆◆◆')

home_path = utils.fetch_experiment_file_path()
annexed_data_paths = [os.path.join(home_path, 'input_data'), os.path.join(home_path, 'output_data')]
data_files = []
selects = []

# 表示するボタン幅のデフォルト値
default_width = '40%'

# データフォルダの読み込み
for data_path in annexed_data_paths:
    data_files += utils.fetch_files(data_path)

data_files.sort()

# upload_selects.valueの値を次のセルで処理の場合分けに利用する
upload_selects = widgets.SelectMultiple(options=data_files, layout=Layout(width=default_width))
button = widgets.Button(description='決　定', layout=Layout(width=default_width))
button.on_click(on_click_callback)

# 「すべてアップロードする」がクリックされたら全ファイルをupload_selectsに追加
button_all = widgets.Button(description='すべてアップロードする', layout=Layout(width=default_width))
button_all.on_click(on_click_callback_ALL)

output = widgets.Output()

display(upload_selects, button, button_all, output)

In [ ]:
import papermill as pm

upload_data_paths = list(upload_selects.value)

index = 1
for save_path in upload_data_paths:
    print(f'保存中です... ({index}/{len(upload_data_paths)})')
    pm.execute_notebook(
        './base_datalad_save_push.ipynb',
        '/home/jovyan/.local/push_log.ipynb',
        parameters = dict(SIBLING_NAME=special_remote_name, SAVE_MESSAGE='[GIN] データを保存', PATH=save_path, TO_GIT=False)
    )
    index += 1

print('データのアップロードが完了しました。')

## 1-B. 実データをオブジェクトストレージから取得する

この手順では、「０．オブジェクトストレージとの接続設定をする」で設定したオブジェクトストレージから、「git-annex」の管理下におかれた実データをダウンロードする際のサポートを提供します。

以下のセルを順番に実行し、表示にしたがって操作することでデータをアップロードできます。

In [ ]:
import os
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display

from scripts import utils

####################################################################################################
########################################## callback functions ######################################
####################################################################################################
def on_click_callback(clicked_button: widgets.Button) -> None:
    with output:
        output.clear_output()
        # セレクタで選択されているアイテムを使う
        print(f'選択されたファイル：{download_selects.value}')
        print('これらのファイルをオブジェクトストレージから取得します。問題ない場合は、次のセルへ進んでください。')
        print('選び直す場合は、再度ボックスから選択し、「決定」ボタンを押してください。')
        
def on_click_callback_ALL(clicked_button: widgets.Button):
    with output:
        print('上のボックスに表示されている、全てのファイルをオブジェクトストレージから取得します。\n問題ない場合は、次のセルへ進んでください。')
        print('選び直す場合は、再度ボックスから選択し、「決定」ボタンを押してください。')
        download_selects.value = data_files

####################################################################################################
####################################################################################################
####################################################################################################

print('どのデータをダウンロードしますか？')
print('　◆◆◆ctrlキー(MAC:commandキー）を押しながらクリックすることで、複数データを選択できます◆◆◆')

home_path = utils.fetch_experiment_file_path()
annexed_data_paths = [os.path.join(home_path, 'input_data'), os.path.join(home_path, 'output_data')]
data_files = []
selects = []

# 表示するボタン幅のデフォルト値
default_width = '40%'

# データフォルダの読み込み
for data_path in annexed_data_paths:
    data_files += utils.fetch_files(data_path)

data_files.sort()
    
# download_selects.valueの値を次のセルで処理の場合分けに利用する
download_selects = widgets.SelectMultiple(options=data_files, layout=Layout(width=default_width))
button = widgets.Button(description='決　定', layout=Layout(width=default_width))
button.on_click(on_click_callback)

# 「すべてアップロードする」がクリックされたら全ファイルをdownload_selectsに追加
button_all = widgets.Button(description='すべてダウンロードする', layout=Layout(width=default_width))
button_all.on_click(on_click_callback_ALL)

output = widgets.Output(layour={'border': '1px solid black'})

display(download_selects, button, button_all, output)

In [ ]:
# Note: 前のセルからdownload_selects.valueを利用している
download_data_paths = list(download_selects.value)

index = 1
for get_path in download_data_paths:
    print(f'データ取得中です... ({index}/{len(download_data_paths)})')
    !datalad get $get_path
    !datalad unlock $get_path
    index += 1

print('実データのダウンロードが完了しました。')